In [1]:
import pandas as pd
import numpy as np

In [5]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/1207623996.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')


In [6]:
def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']

    # Calculate magnitudes along x, y, and z axes
    accel_mag_x_y = np.sqrt(accel_x**2 + accel_y**2)
    accel_mag_z = np.abs(accel_z)
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check if there is significant upward or downward motion along the z-axis
            if accel_z > 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Upward Horizontal Motion"
            elif accel_z < 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Downward Horizontal Motion"
            else:
                motion_type = "Other Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_motion, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_motion.pkl')

# Get the encodings and decodings
encodings = label_encoder_motion.transform(label_encoder_motion.classes_)
decodings = label_encoder_motion.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
smartphone = smartphone.drop(columns = ['activity_type'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

# Group activity_type_full by timestamp and transform it to align with smartphone
activity_type_grouped = activity_type_full.groupby(smartphone['timestamp'])['activity_type'].apply(lambda x: ','.join(x)).reset_index()

# Merge activity_type_grouped with smartphone based on timestamp
smartphone = pd.merge(smartphone, activity_type_grouped, on='timestamp', how='left')

# Print or further process the updated DataFrame
print(smartphone)

smartphone.fillna(0, inplace=True)

# Filter the DataFrame to keep rows where "activity_type" is not equal to 0
merged_data_s_f = smartphone[smartphone['activity_type'] != 0]
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute

merged_data_s_f_filtered = merged_data_s_f[['timestamp', 'activity_type']]

In [ ]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
pivot_data4['timestamp'] = pd.to_datetime(pivot_data4['timestamp'])
# Extract hour and minute from the timestamp and convert to minutes
pivot_data4['timestamp'] = pivot_data4['timestamp'].dt.hour * 60 + pivot_data4['timestamp'].dt.minute
# Merge the dataframes on the 'timestamp' column
pivot_data4_merged = pivot_data4.merge(merged_data_s_f_filtered, on='timestamp', how='left')
pivot_data4_merged.fillna(0, inplace=True)
pivot_data4_merged_filtered = pivot_data4_merged[pivot_data4_merged['activity_type'] != 0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_activities = LabelEncoder()
pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_activities, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_activities.pkl')

# Get unique classes from the LabelEncoder
unique_classes = label_encoder_activities.classes_

# Print unique classes
print("Unique Classes:")
for class_ in unique_classes:
    print(class_)

# Get the encodings and decodings
encodings = label_encoder_activities.transform(label_encoder_activities.classes_)
decodings = label_encoder_activities.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

In [ ]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Downward Horizontal Motion":
        return [13, 20, 21, 3, 6, 18, 4]
    elif motion_type == "Upward Horizontal Motion":
        return [6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Other Moving":
        return [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Turning/Tilting":
        return [11, 17]
    elif motion_type == "Stationary":
        return [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]

# Apply the function to create the predicted_activity column
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [ ]:
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Assuming 'pivot_data4' is your DataFrame
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'motion_encoded', 'timestamp']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(mlb.classes_), activation='sigmoid')  # Sigmoid activation for multi-label classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Convert predictions back to original labels
y_pred_labels = mlb.inverse_transform(y_pred)

# Convert ground truth labels back to original format
y_true_labels = mlb.inverse_transform(y_test)

# Convert ground truth labels to binary array format
y_true_binary = mlb.transform(y_true_labels)

# Print classification report
print(classification_report(y_true_binary, y_pred, target_names=mlb.classes_.astype(str)))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'motion_encoded', 'timestamp']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.multioutput import MultiOutputClassifier

# Initialize a multi-output classifier with GradientBoostingClassifier as the base estimator
multi_gb_model = MultiOutputClassifier(GradientBoostingClassifier())

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Load the previously trained model
model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Retrain the model with class weights
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_binarizer.classes_.astype(str)))

In [ ]:
model = joblib.load("/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras")

X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]

# Define the new target variable (y) using single true labels
y = pivot_data4_merged_filtered['true_motion']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = loaded_model.fit(X_train_scaled, y_train_encoded, epochs=100, batch_size= 45, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Evaluate the retrained model on the test set
test_loss, test_accuracy = loaded_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = loaded_model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)